In [ ]:
# 导入相关包
import os
import pathlib as pl
import pandas as pd
import numpy as np
import re
from io import StringIO
from datetime import datetime,timedelta
import time
from IPython.core.interactiveshell import InteractiveShell
from tqdm.autonotebook import *
import pdfplumber
tqdm.pandas()
InteractiveShell.ast_node_interactivity = "all"
sys.path.append("..")

from tgrocery import Grocery
import jieba

In [ ]:
# 数据准备(train_output文件中格式有点问题，需要提前用excel或者wps打开然后另存为excel文件)
train_outputs = pd.read_excel('../datasets/train_output.xlsx')

# 获取pdf中文字和表格
def extract_pdf_content(pdf_path):
    text_list = []
    table_list = []
    with pdfplumber.open(pdf_path) as pdf:
        for index_page in np.arange(0, len(pdf.pages), 1):
            # 读取多页
            page = pdf.pages[index_page]   # 第n页的信息
            text = page.extract_text()
            text_list.append(text)
            table = page.extract_tables()
            for t in table:
                table_list.append(t)
    return text_list, table_list

def get_dir_file(path):
    '''
    输入文件夹位置，输出整理好的dataframe
    '''
    path_list = os.listdir(path)
    id_list = []
    file_path_list = []
    text_list = []
    table_list = []
    for i in tqdm(path_list):
        if '.PDF' in i:
            file_path = path + i
            id_list.append(int(i.split('.')[0]))
            file_path_list.append(file_path)
            try:
                text_temp, table_temp = extract_pdf_content(file_path)
            except Exception:
                print('此pdf无法读取')
                text_temp, table_temp = [], []
            text_list.append(text_temp)
            table_list.append(table_temp)
            
    df = pd.DataFrame()
    df['sample_id'] = id_list
    df['file_path'] = file_path_list
    df['text'] = text_list
    df['tabel'] = table_list
    df = df.sort_values('sample_id')
    return df

# 文件处理太慢，可持续化保存文件
train_path = '../datasets/train.csv'
if os.path.exists(train_path):
    train_df = pd.read_csv(train_path)
else:
    train_df = get_dir_file('datasets/train_data/')
    train_df.to_csv(train_path,index=False)
    train_df = pd.read_csv(train_path)

test_path =  '../datasets/test.csv'
if os.path.exists(test_path):
    test_df = pd.read_csv(test_path)
else:
    test_df = get_dir_file('datasets/test_data/')
    test_df.to_csv(test_path,index=False)
    test_df = pd.read_csv(test_path)

train_outputs.head(2)
train_df.head(2)
test_df.head(2)

In [ ]:
#text=text.replace(r'[ ]+',' ').replace('\r','^')
def get_title(text):
    global title_num_char
    title_list=[]
    title_type_list=[]
    text_start_iter_list=[]
    text_end_iter_list=[]
 
    for item in title_num_char:
        pattern = re.compile(item+"[0-9A-Za-z\u4e00-\u9fa5 ()\[\]（）【】：:]*?[\^]")
        #pattern = re.compile(item+r"[ ]*?[^ ]+?[ ]")
        #pattern = re.compile(item+"[0-9A-Za-z\u4e00-\u9fa5 ()[]（）【】][]")#*?[\^]
        tmp=pattern.finditer(text)
        for i in tmp:
            title_list.append(i.group())
            text_start_iter_list.append(i.span(0)[0])
            title_type_list.append(1)
            text_end_iter_list.append(i.span(0)[1])

    # for item in title_list:
    for item in s_title_num_char:
        # pattern = re.compile(item+r"[ ]*?[ ]*?[\d][^ ]+?:?[ ]?") 
        # pattern = re.compile(item+"[0-9A-Za-z\u4e00-\u9fa5\^ ]*")
        # pattern = re.compile(item+'[0-9\u4e00-\u9fa5()[]（）【】*?[\^]]')*?[\^]
    #    pattern = re.compile(item+r"[\d]*?\u4e00-\u9fa5+[ ]")
        pattern = re.compile(item+"[0-9A-Za-z\u4e00-\u9fa5 ()\[\]（）【】：:]*?[ ][\^]")
        tmp=pattern.finditer(text)
        for i in tmp:
            title_list.append(i.group())
            text_start_iter_list.append(i.span(0)[0])
            title_type_list.append(1)
            text_end_iter_list.append(i.span(0)[1])

    title_list.append("引言")
    title_type_list.append(1)
    text_start_iter_list.append(0)
    text_end_iter_list.append(0)

    result_df=pd.DataFrame([title_list,title_type_list,text_start_iter_list,text_end_iter_list]).T.sort_values(by=2).reset_index(drop=True)
    # print(result_df)
    return result_df

def get_title_text(text,title_df):
    # print(title_df)
    title_1_df=title_df[title_df[1]==1]
    text_iter_list=[]
    text_list=[]
    # print(title_1_df)
    for iter1,iter2 in title_1_df[[2,3]].values:
        # print(iter1)
        if(len(text_iter_list)!=0):
            text_iter_list.append(iter1)
        text_iter_list.append(iter2)
    # text_iter_list.append(text_iter_list[len(text_iter_list)-1])
    text_iter_list.append(len(text))
    for index in range(int(len(text_iter_list)/2)):
        text_list.append(text[text_iter_list[2*index]:text_iter_list[2*index+1]])
    
    title_1_df[4]=text_list

    return title_1_df.reset_index(drop=True)

#from fuzzywuzzy import fuzz
def judge_title(sample_id=0,text=r"test\n"):
    # print(text)
    text=text.replace(r"\n","^").replace(r'[ ]+',' ')
    title_df=get_title(text)
    title_df["sample_id"]=[sample_id for x in range(title_df.shape[0])]
    # print(title_df)`
    title_1_df=get_title_text(text,title_df)[["sample_id",0,1,2,3,4]]

    

    global val_df
    global train_outputs
    val_true_name=train_outputs[train_outputs["sample_id"]==sample_id]["理财产品名称"]
    
    index=0
    neg_index=[]
    for title_des in title_1_df[0].values:
        for item in title_neg_words:
            if re.search(item,title_des) is not None:
                neg_index.append(index)
                break
        index+=1


    return title_1_df.drop(neg_index)
    # print(title_list)

def get_judge_title_result(val_df):

    judge_title_result=None


    for sample_id,text in tqdm(val_df[["sample_id","text"]].values):
        # print(sample_id)
        # print(text)
        judge_title_result= judge_title(sample_id,text) if judge_title_result is None else pd.concat([judge_title_result,judge_title(sample_id,text)])

        judge_title_result["sample_id"]=judge_title_result["sample_id"].astype(str)        
    return judge_title_result

title_num_char=["一、","二、","三、","四、","五、","六、","七、","八、","九、","十、","十一、","十二、","十三、","十四、","十五、"]
s_title_num_char=["（一）","（二）","（三）","（四）","（五）","（六）","（七）","（八）","（九）","（十）","（十一）","（十二）","（十三）","（十四）","（十五）"]
s_title_num_char.extend(["[(]一[)]","[(]二[)]","[(]三[)]","[(]四[)]","[(]五[)]","[(]六[)]","[(]七[)]","[(]八[)]","[(]九[)]","[(]十[)]","[(]十一[)]","[(]十二[)]","[(]十三[)]","[(]十四[)]","[(]十五[)]"])

title_pos_words=[]
title_neg_words=["备查","日前","过去","履行","审批","程序","风险","措施","影响","累计","赎回","到期","截至","意见","十二个月内","公告前","报备文件","前期"]

val_judge_title_result=get_judge_title_result(val_df)
train_judge_title_result=get_judge_title_result(train_df)
#test_judge_title_result=get_judge_title_result(test_df)
# judge_title_result.to_excel("训练集段落标题分类结果.xlsx",index=None)

In [ ]:
def get_content_df(val_judge_title_result):
    sample_id=val_judge_title_result['sample_id'].drop_duplicates() 
    text_list=[]
    text_sampleid_list=[]
    content_df=pd.DataFrame()
    for i in tqdm(sample_id.head(10)):
        text_df=val_judge_title_result[val_judge_title_result['sample_id']==i]
    #  text_df.iloc[0,5]
        #text_df
        n=len(text_df)
        text_sampleid_list.append([i])
        text=''
        for y in range(0,n):
            text=text+text_df.iloc[y,1]+text_df.iloc[y,5]
            text=re.sub("[ ]+"," ",text)
        #text
        text_list.append(text)
    content_df['sample_id']=text_sampleid_list
    content_df['text']=text_list
    return content_df

val_content_df=get_content_df(val_judge_title_result)
test_content_df=get_content_df(test_content_df)

In [ ]:
def get_mc(text):
    # my_list=cut_list(text)
    mc_list=[]
    my_list=cut_list(text)
    if my_list==0:
        return 
    else:
        my_list=cut_list(text)

        for i in my_list:
            this_str=i[0]
            if "名" not in this_str or "受托方" in this_str or "公司名称" in this_str:
                continue
            index=this_str.index("：")
            this_str=this_str[index+1:].replace('。','').replace('；','').replace(';','').replace('、','')
            mc_list.append(this_str)
            # print(mc_list) 
    if mc_list ==[]:
        return
    return mc_list
